In [ ]:
import torch
from vectorhash import build_vectorhash_architecture
from smoothing import RatSLAMSmoothing, ArgmaxSmoothing, PolynomialSmoothing, SoftmaxSmoothing, SequentialSmoothing
from shifts import RollShift
from data_utils import load_mnist_dataset, prepare_data
from graph_utils import plot_imgs_side_by_side
from matplotlib import pyplot as plt
device = "cuda"

In [ ]:
N = 200
dataset = load_mnist_dataset()
obs, noisy_obs = prepare_data(dataset, num_imgs=N, device=device)

In [ ]:
smoothings = [
    # ArgmaxSmoothing(),
    # PolynomialSmoothing(k=1.5),
    # SoftmaxSmoothing(T=0.01),
    # SoftmaxSmoothing(T=0.03),
    # RatSLAMSmoothing(device=device),
    # RatSLAMSmoothing(device=device, sigma_xy=0.3, sigma_theta=0.3, inhibition_constant=0.00, delta_gamma=2),
    SequentialSmoothing([
        RatSLAMSmoothing(device=device),
        SoftmaxSmoothing(T=0.0002)
    ])E
]

obs_results = torch.ones(len(smoothings), N, 3, device=device)
noisy_obs_results = torch.ones(len(smoothings), N, 3, device=device)
noise_results = torch.ones(len(smoothings), 3, device=device)
noise = torch.randn(28 * 28, device=device)

limits=torch.Tensor([1,1,1])

for i, method in enumerate(smoothings):
    model = build_vectorhash_architecture(
        shapes=[(5,5,5),(8,8,8)],
        N_h=1200,
        input_size=28 * 28,
        initalization_method="by_sparsity",
        percent_nonzero_relu=0.2,
        device=device,
        smoothing=method,
        # smoothing=ArgmaxSmoothing(),
        shift=RollShift(),
        relu=True,
        epsilon_hs=0.01,
        epsilon_sh=0.01,
    )
    for j in range(N):
        model.hippocampal_sensory_layer.learn(
            model.scaffold.hippocampal_from_grid(model.scaffold.G[i])[0], obs[i]
        )

    for k in range(N):
        obs_certainty, noisy_obs_certainty = [
            model.scaffold.estimate_certainty(
                g=model.scaffold.denoise(
                    model.scaffold.grid_from_hippocampal(
                        model.hippocampal_sensory_layer.hippocampal_from_sensory(o)
                    )
                )[0],
                limits=limits
            )
            for o in [obs[k], noisy_obs[k]]
        ]
        obs_results[i, k] = obs_certainty
        noisy_obs_results[i, k] = noisy_obs_certainty
    noise_certainty = model.scaffold.estimate_certainty(
        g=model.scaffold.denoise(
            model.scaffold.grid_from_hippocampal(
                model.hippocampal_sensory_layer.hippocampal_from_sensory(noise)
            )
        )[0],
        limits=limits
    )
    noise_results[i] = noise_certainty

In [ ]:
# obs_results = torch.ones(len(smoothings), N, 3, device=device)
# noisy_obs_results = torch.ones(len(smoothings), N, 3, device=device)

fig, ax = plt.subplots(figsize=(10, 6))

k = 40
dim = 2

for i, method in enumerate(smoothings):
    # if not isinstance(method, SoftmaxSmoothing):
    #     continue


    # if not isinstance(method, PolynomialSmoothing):
    #     continue

    # if not isinstance(method, RatSLAMSmoothing):
    #     continue

    # if not isinstance(method, SequentialSmoothing):
        # continue
    ax.scatter(
        torch.arange(0, k), obs_results[i, :k, dim].cpu(), label=f"{method}, obs", s=5
    )
    ax.scatter(
        torch.arange(0, k),
        noisy_obs_results[i, :k, dim].cpu(),
        label=f"{method}, noisy obs",
        s=5,
    )
    ax.axhline(y=noise_results[i, dim].cpu(), label=f"noise {method}", linestyle=":")

ax.set_ylim(0, 1.25)
ax.set_xlim(0, k)
ax.legend(
    bbox_to_anchor=(1.3, -0.4), loc="lower right", bbox_transform=plt.gcf().transFigure
)
ax.set_xlabel("image number")
ax.set_ylabel("certainty")